In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [2]:
# Require 3: return a dataframe that includes the monthly mean climate data for a given county and a given period

# The code of a county can be found by searching "FIPS county code" in Google, which was 5-digit id.
# Here is a link as well. https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt

Countycode = "31109"        # code of Lancaster County, NE used in this project
StartDate = "2018-01-01"                   # the given start date in this project.
EndDate = "2018-12-31"                     # the given start date in this project.

url = f"http://data.rcc-acis.org/StnMeta?county={Countycode}&output=csv"
url

'http://data.rcc-acis.org/StnMeta?county=31109&output=csv'

In [3]:
Meta_county = pd.read_csv(url, sep = ",")
Meta_county

,252935,FIRTH,NE,-96.608060,40.535000,1340
0,253523,HALLAM 3 W,NE,-96.85000,40.53333,1401.0
1,253923,HOLLAND,NE,-96.58333,40.58333,1352.0
2,253829,HICKMAN 2 SW,NE,-96.63333,40.58333,NaN
3,256948,PRINCETON,NE,-96.70000,40.56667,1430.0
4,253524,HALLAM 2 N,NE,-96.78333,40.56667,1450.0
...,...,...,...,...,...,...
136,US1NELA0052,LINCOLN 1.2 ENE,NE,-96.66780,40.82411,1207.0
137,US1NELA0055,LINCOLN 4.4 ESE,NE,-96.61607,40.78303,1289.0
138,US1NELA0054,LINCOLN 4.6 N,NE,-96.69430,40.88230,1210.0
139,US1NELA0053,HICKMAN 1.8 NNE,NE,-96.62077,40.64552,1369.0


In [4]:
# Defining the header of the dataframe
columns = ["StationID", "StationName", "State", "Lon", "Lat", "Elevation"]
Meta_county = pd.read_csv(url, sep = ",", names = columns)
Meta_county

,StationID,StationName,State,Lon,Lat,Elevation
0,252935,FIRTH,NE,-96.60806,40.53500,1340.0
1,253523,HALLAM 3 W,NE,-96.85000,40.53333,1401.0
2,253923,HOLLAND,NE,-96.58333,40.58333,1352.0
3,253829,HICKMAN 2 SW,NE,-96.63333,40.58333,NaN
4,256948,PRINCETON,NE,-96.70000,40.56667,1430.0
...,...,...,...,...,...,...
137,US1NELA0052,LINCOLN 1.2 ENE,NE,-96.66780,40.82411,1207.0
138,US1NELA0055,LINCOLN 4.4 ESE,NE,-96.61607,40.78303,1289.0
139,US1NELA0054,LINCOLN 4.6 N,NE,-96.69430,40.88230,1210.0
140,US1NELA0053,HICKMAN 1.8 NNE,NE,-96.62077,40.64552,1369.0


In [5]:
# The functions in the previous file was needed.

def URLGenerate(Station, StartDate, EndDate):
    URLgenerated = f"http://data.rcc-acis.org/StnData?sid={Station}&sdate={StartDate}&edate={EndDate}&elems=1,2,43,4&output=csv"
    
    return URLgenerated


# Function used to read the URL that specifified a station and a given period including start date and end date.
def ReadURLData(Station, StartDate, EndDate):
    URL = URLGenerate(Station, StartDate, EndDate)
    df = pd.read_csv(URL, sep = ",", skiprows = 1, header = None)
    df.columns = ["Date", "MaxTemp", "MinTemp", "AverTemp", "Preci"]
    df["Station"] = Station
    df.Date = pd.to_datetime(df.Date)
    df.MaxTemp = pd.to_numeric(df.MaxTemp, errors = "coerce", downcast = "float")
    df.MinTemp = pd.to_numeric(df.MinTemp, errors = "coerce", downcast = "float")
    df.AverTemp = pd.to_numeric(df.AverTemp, errors = "coerce", downcast = "float")
    df.Preci = pd.to_numeric(df.Preci, errors = "coerce")
    df.set_index("Date",inplace = True)
    
    return df


In [6]:
all_df = pd.DataFrame(columns=["MaxTemp", "MinTemp", "AverTemp", "Preci", "Station"])
for index, row in Meta_county.iterrows():
    # print(row.StationID)
    try:
        each_station = ReadURLData(row.StationID, StartDate, EndDate)
        all_df = pd.concat([all_df, each_station])
    except:
        continue
all_df

,MaxTemp,MinTemp,AverTemp,Preci,Station
2018-01-01,NaN,NaN,NaN,NaN,252935
2018-01-02,NaN,NaN,NaN,NaN,252935
2018-01-03,NaN,NaN,NaN,NaN,252935
2018-01-04,NaN,NaN,NaN,NaN,252935
2018-01-05,NaN,NaN,NaN,NaN,252935
...,...,...,...,...,...
2018-12-27,NaN,NaN,NaN,NaN,US1NELA0057
2018-12-28,NaN,NaN,NaN,NaN,US1NELA0057
2018-12-29,NaN,NaN,NaN,NaN,US1NELA0057
2018-12-30,NaN,NaN,NaN,NaN,US1NELA0057


In [7]:
monthlyCountyMean = all_df.resample("M").mean()
monthlyCountyMean

,MaxTemp,MinTemp,AverTemp,Preci
2018-01-31,34.012905,12.658065,23.335484,0.020870
2018-02-28,34.221428,12.642858,23.432142,0.035883
2018-03-31,51.232258,29.238710,40.235485,0.097737
2018-04-30,57.466667,29.853333,43.660000,0.027034
2018-05-31,81.593552,56.174194,68.883873,0.114284
2018-06-30,86.706665,65.046669,75.876663,0.298700
2018-07-31,85.941933,63.799999,74.870964,0.222770
2018-08-31,84.627449,62.764706,73.696075,0.216845
2018-09-30,77.933334,57.799999,67.866669,0.374116
2018-10-31,61.458065,38.793549,50.125805,0.159743
